**Лабораторный практикум по курсу «Распознавание диктора», Университет ИТМО, 2021**		

**Лабораторная работа №1. Информативные признаки речевых сигналов: извлечение признаков**

**Цель работы:** изучение процедуры построения информативных акустических признаков для речевых сигналов.

**Содержание лабораторной работы**

1. Выполнить процедуру высокочастотной фильтрации (преэмфазис) речевого сигнала.

2. Вычислить акустические признаки разных видов.

3. Выполнить локальные центрирование и масштабирование акустических признаков.

4. Построить распределение первых 3 компонент полученных акустических признаков для мужских и женских голосов.

In [1]:
# Import of modules
import os
import sys

sys.path.append(os.path.realpath('..'))

from common import download_dataset, extract_dataset

**1. Подготовка данных для обучения и тестирования детектора речевой активности**

В ходе выполнения лабораторной работы необходимы данные для осуществления процедуры вычисления акустических признаков. Возьмём в качестве этих данных несколько звукозаписей голосов людей мужского и женского пола, сохраненных в формат *wav*, выбранных из корпуса [VoxCeleb1 test set](https://thor.robots.ox.ac.uk/~vgg/data/voxceleb/vox1a/vox1_test_wav.zip). Данный корпус содержит 4,874 звукозаписи (частота дискретизации равна 16кГц) 40 дикторов мужского и женского пола, разговаривающих на английском языке.

В рамках настоящего пункта требуется выполнить загрузку и распаковку звуковых wav-файлов из корпуса VoxCeleb1 test set.

![Рисунок 1](https://analyticsindiamag.com/wp-content/uploads/2020/12/image.png "VoxCeleb. Крупномасштабная аудиовизуальная база данных человеческой речи.")

In [2]:
# Download VoxCeleb1 (test set)
with open('../data/lists/datasets.txt', 'r') as f:
    lines = f.readlines()

download_dataset(lines, user='voxceleb1902', password='nx0bl2v2', save_path='../data')

Checksum successful vox1_test_wav.zip.


In [3]:
# Extract VoxCeleb1 test set
extract_dataset(save_path='../data/voxceleb1_test', fname='../data/vox1_test_wav.zip')

Extracting of ../data/vox1_test_wav.zip is successful.


**2. Выполнить процедуру высокочастотной фильтрации (преэмфазис) речевого сигнала**

Можно добавить краткую теорию про преэмфазис. Краткое описание того, что нужно сделать.

**3. Вычислить акустические признаки разных видов**

Можно добавить краткую теорию про вычисление логарифмов энергий на выходе мел-банка фильтров размерности 40 и мел-частотные кепстральные коэффициенты размерности 23. Краткое описание того, что нужно сделать.

**4. Выполнить локальные центрирование и масштабирование акустических признаков**

Можно добавить краткую теорию про выполнения процедур нормализации и масштабирования. Краткое описание того, что нужно сделать.

**5. Построить распределение первых 3 компонент полученных акустических признаков для мужских и женских голосов**

Краткое описание того, что нужно сделать.

**6. Контрольные вопросы**

1. Какие способы представления сигналов существуют?

2. Что такое спектр Фурье (амплитудный и фазовый)?

3. Что такое оконное преобразование Фурье?

4. Что такое спектрограмма?

5. Как выполнить процедуру преэмфазиса?

6. Описать процедуру вычисления акустических признаков.

7. Для каких целей выполняются процедуры нормализации и масштабирования акустических признаков?

**7. Список литературы**

1. Hayes M.H. Schaum’s outlines of digital signal processing. McGraw-Hill, 2011 ([ссылка](http://index-of.co.uk/DSP-Collection/Hayes,%20M.H.,%20Schaum's%20Outline%20of%20Digital%20Signal%20Processing,%20McGraw-Hill,%201999.pdf)).

2. Beigi H. Fundamentals of speaker recogniton. Springer, 2011 ([ссылка](https://www.springer.com/gp/book/9780387775913)).

### Лабораторная работа №1. Информативные признаки речевых сигналов: извлечение признаков
**Описание**:
В рамках данной лабораторной работы вам предстоит познакомиться с процедурами предобработки речевых сигналов и извлечения информативных признаков. В данной работе вам предлагается научиться извлекать

- Mel Filter Banks - кратковременные энергии мелчастотных полос
- Mel Frequency Cepstral Coeffitients - Мел частотные кепстральные коэффициенты

Признаки должны совпадать с соответствующими признаками, которые извлекаются в **torchaudio** (https://pytorch.org/audio/stable/transforms.html)

**Данные**:
В качестве данных для проведения исследования предлагается взять базу VoxCeleb1: http://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox1.html.

**Задания**:
1. Сделать preemphasis (фильтрация верхних частот). Проанализировать амлитудно-частотные спектры исходного и обработанного речевых участков сигналов
2. Вычислить акустические признаки разных видов: FBs40 и MFCCs23.
3. Выполнить локальное центрирование и масштабирование признаков (нормализация MVN).
4. Построить распределение первых 3 компонент полученных признаков по всей базе отдельно по гендерам

### 1. Preemphasis preprocessing

In [2]:
from audiofeatures import Audiofeatures
processed_waveform = Audiofeatures.preemphasis(waveform)

NameError: name 'waveform' is not defined

### 2. Mel Filter Banks computing

In [3]:
MFB40_FeaturesExctractor = Audiofeatures(feats_type='MFB40')
MFB40 = MFB40_FeaturesExctractor.extract('wav_path')

NotImplementedError: 

### 3. Mel Frequency Cepstral Coeffitiens computing

In [ ]:
MFCC_FeaturesExctractor = Audiofeatures(feats_type='MFCC')
MFCC = MFCC_FeaturesExctractor.extract('wav_path')